### Init Context

In [1]:
from thetaray.api.context import init_context
import datetime
import yaml

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_pay_proc/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']
context = init_context(execution_date=datetime.datetime(1970, 1, 1),
                       spark_conf=spark_config,
                       spark_master='local[*]',
                       delete_unused_columns=True)

2025-07-22 12:54:23,407:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-07-22 12:54:23,413:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_remittance_customer_insights, but data encryption is disabled in deployment
2025-07-22 12:54:23,422:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-07-22 12:54:23,446:WARNING:thetaray.common.logging:Encryption is enabled on dataset wrangling, but data encryption is disabled in deployment
2025-07-22 12:54:23,448:WARNING:thetaray.common.logging:Encryption is enabled on dataset party_wrangling, but data encryption is disabled in deployment
2025-07-22 12:54:23,449:WARNING:thetaray.common.logging:Encryption is enabled on dataset customer_insights, but data encryption is disabled in deployment
2025-07-22 12:

### Imports

In [2]:
from thetaray.api.dataset import dataset_functions
from thetaray.api.evaluation import fit_on_worker
from thetaray.api.histograms import save_histograms
from thetaray.api.evaluation.preprocess.features_extractor import FeaturesExtractor
from thetaray.api.models import save_model
from thetaray.api.anomaly_detection import ThetaRayDetector
from thetaray.api.evaluation.preprocess.numeric_features import NumericFeaturesTransformer
from thetaray.common.data_environment import DataEnvironment

import mlflow
from pyspark.sql import functions as f
import pandas as pd

from domains.demo_pay_proc.datasets.customer_monthly import customer_monthly_dataset as input_dataset
from domains.demo_pay_proc.evaluation_flows.ef import evaluation_flow as ef

### Load data

In [3]:
data = dataset_functions.read(context, input_dataset().identifier, data_environment=DataEnvironment.PUBLIC)
data = data.orderBy([f.hash('customer_id'), 'year_month']).filter(f.col('year_month') >= pd.to_datetime('2024-11-1'))
data_pd = data.toPandas()
data_pd

,txn_count,reversals,pipe_account_behaviour_score,year_month_str,avg_txn_amt_ratio,terminals,new_terminal_ratio,rare_currency_pct,domestic_ratio_change,avg_amount,year_month,rare_currency_count,customer_name,customer_id,reversal_ratio,txn_count_ratio,tr_timestamp
0,2527,8,0.1105,2024-11,0.0005,1,0.0,0.0,0.8144,109.902014,2024-11-01,1,Jensen-Hill,MERCHANT_064,0.5000,0.5189,2025-07-22 12:49:31.373868
1,2105,6,0.1416,2024-12,0.0011,3,0.5,0.0,0.7967,109.979078,2024-12-01,1,Jensen-Hill,MERCHANT_064,0.0000,0.1200,2025-07-22 12:49:31.373868
2,1865,4,0.1785,2025-01,0.0000,1,0.0,0.0,0.8021,109.242767,2025-01-01,1,Jensen-Hill,MERCHANT_064,0.0000,0.0000,2025-07-22 12:49:31.373868
3,1109,3,0.1039,2025-02,0.0054,1,0.0,0.0,0.7890,110.367412,2025-02-01,1,Jensen-Hill,MERCHANT_064,0.0000,0.0000,2025-07-22 12:49:31.373868
4,2028,6,0.1211,2025-03,0.0000,2,0.0,0.0,0.8042,109.702184,2025-03-01,1,Jensen-Hill,MERCHANT_064,0.2000,0.1389,2025-07-22 12:49:31.373868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,2541,6,0.1795,2025-03,0.0000,1,0.0,0.0,0.8127,109.652534,2025-03-01,1,Lee PLC,MERCHANT_083,0.0000,0.2170,2025-07-22 12:49:31.373868
806,1440,4,0.1914,2025-04,0.0000,2,0.0,0.0,0.8292,109.942250,2025-04-01,0,Lee PLC,MERCHANT_083,0.0000,0.0000,2025-07-22 12:49:31.373868
807,2441,6,0.1118,2025-05,0.0000,1,0.0,0.2,0.8013,109.407386,2025-05-01,1,Lee PLC,MERCHANT_083,0.0909,0.2661,2025-07-22 12:49:31.373868
808,2073,7,0.1971,2025-06,0.0035,1,0.0,0.2,0.8037,110.156097,2025-06-01,1,Lee PLC,MERCHANT_083,0.3125,0.0227,2025-07-22 12:49:31.373868


### Model Training

In [4]:
requested_features = ['txn_count_ratio', 
                      'reversal_ratio', 
                      'rare_currency_pct', 
                      'new_terminal_ratio', 
                      'avg_txn_amt_ratio', 
                      'domestic_ratio_change', 
                      'pipe_account_behaviour_score']
nft = NumericFeaturesTransformer(features=requested_features, strategy='constant', fill_value=0.0)
fu = FeaturesExtractor([nft])
trd = ThetaRayDetector(algo_type=['Ny', 'NF', 'GC'],
                       learning_method=1,
                       normalization_type=1,
                       Fusion_threshold=0.5,
                       Rating_percentile=5.0,
                       set_zero_rating=1)

with mlflow.start_run(nested=True):
    feature_extraction_model = fit_on_worker(fu.fit, X=data_pd)
    save_model(ef().evaluation_steps[0].feature_extraction_model.name, feature_extraction_model, tags=ef().evaluation_steps[0].feature_extraction_model.tags, data_environment=DataEnvironment.PUBLIC)
    detection_model = fit_on_worker(trd.fit, X=feature_extraction_model.transform(data_pd))
    save_model(ef().evaluation_steps[0].detection_model.name, detection_model, tags=ef().evaluation_steps[0].detection_model.tags, data_environment=DataEnvironment.PUBLIC)
    save_histograms(context, data_pd, requested_features)

2025-07-22 12:55:02.000048: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 12:55:02.027523: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


##################################################
## Migrating to ModularSystem (AnomalyDetector) ##
##################################################
{'PREPROCESSOR': {'DataScaler': {'Normalization_type': 1}, 'DataStandardizer': {'data_type': 'float64'}, 'LabelsStandardizer': {'labels_type': 'int8'}}, 'ALGO': {'TrainingSplitter': {'Method': None, 'Fusion_sampling': {'q': 2, 'lu_data_per': 30, 'lu_acc_rate': 0.95, 'display': 2, 'GEV_size_of_batches': 200, 'percentile': 0.01, 'threshold_type': 0}}, 'Thread': {'mode': 1, 'max_wrk': 'None'}, 'Algos_to_run': ['Ny', 'NF', 'GC'], 'Algos_parameters': {'Ny': {'max_depth': 9, 'epsilon': 0, 'number_of_eigenvalues_EMD': 20, 'kernel_power': 1, 'datasample': 20000, 'prediction_batch_size': 100000, 'gcsmpl_trunc_th': 0.1353, 'gcsmpl_max_size': 10000, 'gcsmpl_max_dictionary_size': 10000, 'gcsmpl_seed': 12347, 'gcsmpl_knn_k': 25, 'gcsmpl_min_sq_col_nrm': -1, 'gcsmpl_sym': 1, 'testing_extension_condition_number': 50000, 'diffusion_start_eig_vector_kr

2025/07/22 12:55:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/07/22 12:55:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-07-22 12:55:10,760:INFO:botocore.credentials:Found credentials in environment variables.
2025/07/22 12:55:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: demo_pay_proc_fe, version 17
2025-07-22 12:55:11.686120: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 12:55:11.713387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use a

🏃 View run aged-horse-213 at: https://mlflow:5000/#/experiments/0/runs/3996361e350d41a0a298f4437bd7d325
🧪 View experiment at: https://mlflow:5000/#/experiments/0


In [5]:
context.close()